In [52]:
import pyodbc
import json
import time
import requests
import urllib
import os
from requests.auth import HTTPBasicAuth
import warnings
import pandas as pd
#from utilities import compare
#from multiprocessing import Pool
warnings.filterwarnings("ignore")
beginTime = time.time()

prod_server='PSQLA012'
test_server= 'QSQLA012'
database='Centripetus'
username='cardio'
password='access'

prdcolumnConnection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+prod_server+';DATABASE='+database+';UID='+username+';PWD='+ password,autocommit=True)
tstcolumnConnection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+test_server+';DATABASE='+database+';UID='+username+';PWD='+ password,autocommit=True)
prdcolumnCursor = prdcolumnConnection.cursor()
tstcolumnCursor = tstcolumnConnection.cursor()


compare_sql = """
select t.name,
       c.name,
	   system_type_id,
	   max_length,
	   precision,
	   scale
from sys.tables t join sys.columns c on t.object_id = c.object_id
where t.name like 'PAC%' and not(t.name like 'PAC3%')
"""


prdcolumnCursor.execute(compare_sql)    
prdcolumnResult = prdcolumnCursor.fetchall()
prdcolumnResultdf = pd.DataFrame.from_records(prdcolumnResult,columns=['table_name','column_name','type_id','maxlength','precision','scale_num'])

print(prdcolumnResultdf)    


tstcolumnCursor.execute(compare_sql)    
tstcolumnResult = tstcolumnCursor.fetchall()
tstcolumnResultdf = pd.DataFrame.from_records(tstcolumnResult,columns=['table_name','column_name','type_id','maxlength','precision','scale_num'])

print(tstcolumnResultdf)   


results = pd.merge(tstcolumnResultdf, prdcolumnResultdf, how='left', on=['table_name', 'column_name'], left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_tst', '_prd'), copy=True, indicator=False,
         validate=None)

results.to_csv(f'{database}_schema_compare.csv')

        table_name         column_name  type_id  maxlength  precision  \
0    PACarrhythMed        ArrhythMedID       56          4         10   
1    PACarrhythMed      PACencounterID       56          4         10   
2    PACarrhythMed      ArrhythMedType       56          4         10   
3    PACarrhythMed  ArrhythMedTypeSpec      231        100          0   
4    PACarrhythMed     ArrhythMedStart       56          4         10   
..             ...                 ...      ...        ...        ...   
276      PACvenous        VenLineVenue       56          4         10   
277      PACvenous        VenLineStart       56          4         10   
278      PACvenous      VenLineStartDt       40          3         10   
279      PACvenous          VenLineEnd       56          4         10   
280      PACvenous        VenLineEndDt       40          3         10   

     scale_num  
0            0  
1            0  
2            0  
3            0  
4            0  
..         ...  
276 